In [1]:
%pip install /Users/lion/lionagi

Processing /Users/lion/lionagi
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached tiktoken-0.7.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached pydantic-2.7.2-py3-none-any.whl.metadata (108 kB)
  Using cached aiocache-0.12.2-py2.py3-none-any.whl.metadata (8.3 kB)
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
  Using cached yarl-1.9.4-cp312-cp312-macosx_11_0_

In [2]:
import lionagi as li

In [3]:
from pathlib import Path

data_path = Path.cwd() / "lionagi_data"  # Path to the data directory

### Prepare Data

In [4]:
# load files from directory
docs = li.load(input_dir=data_path, recursive=True, required_exts=[".py"])
docs = [doc for doc in docs if len(doc.content) > 100]

In [5]:
# # chunk
# pile = li.chunk(docs=docs, chunk_size=2000, overlap=0.1)

# embed
# await pile.embed_pile()

# # save
# pile.to_csv("lionagi_embedding.csv")

In [6]:
# reload pile from saved csv
pile = li.pile(csv_file="lionagi_embedding.csv")

### Construct Workflow

In [7]:
instruction = """
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name of a variable,
function, class, or module is used but not present in context,
you must check with the query engine. make sure to cross 
reference the code with the query engine to ensure the 
documentation is accurate.
"""

In [8]:
from PROMPTS import sys_prompt  # put your system prompt here

model = li.iModel(
    model="gpt-4o",
    provider="openai",
    interval_tokens=1_000_000,
    interval_requests=1_000,
    interval=60,
)

tools = pile.as_query_tool(
    name="qa_lionagi",
    guidance="Perform query to a QA bot",
    query_description="a term/phrase to lookup or a question to answer",
)

/Users/lion/Library/Mobile Documents/com~apple~CloudDocs/GitHub/gitco/test_v02/PROMPTS.py:1: SyntaxWarning: invalid escape sequence '\m'
  sys_prompt = """


Module llama_index or attribute None not found. Installing llama-index...
  Using cached llama_index-0.10.39-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.2.5-py3-none-any.whl.metadata (678 bytes)
  Using cached llama_index_cli-0.1.12-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.10.39.post1-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.1.10-py3-none-any.whl.metadata (604 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.1.6-py3-none-any.whl.metadata (3.8 kB)
  Using cached llama_index_legacy-0.9.48-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_llms_openai-0.1.21-py3-none-any.whl.metadata (559 bytes)
  Using cached llama_index_multi_modal_llms_openai-0.1.6-py3-none-any.whl.metadata (677 bytes)
  Using cached llama_index_program_openai-0.1.6-py3-none-any.whl.metadata (715 bytes)
  Using cached llama_index_question_gen_openai-0.1.3-py3-none-any.whl.metadata (785 bytes)
 

In [9]:
# await pile.query_pile("what is a pile in lionagi?")

In [14]:
branch = li.Branch(system=sys_prompt, tools=tools, imodel=model)

form = await branch.direct(
    instruction=instruction,
    context=docs[83].content,
    allow_action=True,
    allow_extension=True,
    verbose=True,
    max_extensions=2,
    retries=3,  # sometimes the model may fail to generate a valid response or refuse to take actions
)

Chatting with model...
Found action requests in model response. Processing actions...
Actions processed!

Found extension requests in model response.
------------------- Processing extension No.1 -------------------
Chatting with model...
Analyzing action responses and generating answer...
------------------- Extension completed -------------------

Analyzing action responses and generating answer...

--------------------------------------------------------------
Directive successfully completed!


In [15]:
branch.to_df()

,ln_id,message_type,timestamp,role,content,metadata,sender,recipient
0,018e68d2a4d030515c50123fd7165a10,System,2024-05-28T21:46:35.441626,system,"{'system_info': ' you are a helpful assistant,...",{'last_updated': {'recipient': '2024-05-28T21:...,system,ae5ed231af63915188044f18c8194294
1,a34732b8755be3e8abf1fdcf095288fd,Instruction,2024-05-28T21:46:35.442985,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-28T21:46:...,user,ae5ed231af63915188044f18c8194294
2,534719e39ce75a0d7fdad2ae56b82189,AssistantResponse,2024-05-28T21:46:42.030346,assistant,"{'assistant_response': '```json { ""answer"": ...",{'last_updated': {'sender': '2024-05-28T21:46:...,ae5ed231af63915188044f18c8194294,user
3,daf681b0a3a6e8ffba399eee7a9a864c,ActionRequest,2024-05-28T21:46:42.035028,assistant,"{'action_request': {'function': 'qa_lionagi', ...",{'last_updated': {'function': '2024-05-28T21:4...,ae5ed231af63915188044f18c8194294,ac064e0c83163b39f989e5fd616710e1
4,b3e04159d3481c1f7982e09e42bd73a8,ActionRequest,2024-05-28T21:46:42.035496,assistant,"{'action_request': {'function': 'qa_lionagi', ...",{'last_updated': {'function': '2024-05-28T21:4...,ae5ed231af63915188044f18c8194294,ac064e0c83163b39f989e5fd616710e1
5,6e42ed404ae2879b868be2938ad51789,ActionRequest,2024-05-28T21:46:42.035727,assistant,"{'action_request': {'function': 'qa_lionagi', ...",{'last_updated': {'function': '2024-05-28T21:4...,ae5ed231af63915188044f18c8194294,ac064e0c83163b39f989e5fd616710e1
6,01662558346028730684d1b85c2fa989,ActionResponse,2024-05-28T21:46:44.816031,assistant,"{'action_response': {'function': 'qa_lionagi',...",{'last_updated': {'function': '2024-05-28T21:4...,ac064e0c83163b39f989e5fd616710e1,ae5ed231af63915188044f18c8194294
7,7c2a816d9bc496492a3ec1ffa620d62d,ActionResponse,2024-05-28T21:46:44.816316,assistant,"{'action_response': {'function': 'qa_lionagi',...",{'last_updated': {'function': '2024-05-28T21:4...,ac064e0c83163b39f989e5fd616710e1,ae5ed231af63915188044f18c8194294
8,d7294f43e994f836a7ae8277753ff36e,ActionResponse,2024-05-28T21:46:44.816413,assistant,"{'action_response': {'function': 'qa_lionagi',...",{'last_updated': {'function': '2024-05-28T21:4...,ac064e0c83163b39f989e5fd616710e1,ae5ed231af63915188044f18c8194294
9,1856db78ad1a954afe2ac9ae6cd46a37,Instruction,2024-05-28T21:46:44.816977,user,{'instruction': ' ## Task Instructions...,{'last_updated': {'sender': '2024-05-28T21:46:...,user,ae5ed231af63915188044f18c8194294


In [16]:
form.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: 
write a good API documentation for this code, must use 
query engine to check meanings of related code concepts 
to accurately describe, for example if a name of a variable,
function, class, or module is used but not present in context,
you must check with the query engine. make sure to cross 
reference the code with the query engine to ensure the 
documentation is accurate.

- Additional context: """
Copyright 2024 HaiyangLi
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
"""
# lionagi/core/session/directive_mixin.py
from lionagi.libs.ln_convert import strip_lower
from lionagi.core.unit import Unit

class DirectiveMixin:
    """
    Mixin class for handling chat interactions within the directive framework.
    """
    async def chat(
        self,
        instruction,  # additional instruction
        context=None,  # context to perform the instruction on
        system=None,  # optionally swap system message
        sender=None,  # sender of the instruction, default "user"
        recipient=None,  # recipient of the instruction, default "branch.ln_id"
        branch=None,
        requested_fields=None,  # fields to request from the context, default None
        form=None,  # form to create instruction from, default None,
        tools=False,  # the tools to use, use True to consider all tools, no tools by default
        invoke_tool=True,  # whether to invoke the tool when function calling, default True
        return_for...

**reason**: 
 Let's think step by step. To accurately document the provided code, we need to understand the specific meanings of 'directive framework', 'Unit' class, and 'strip_lower' function as they are not fully explained in the provided context. Querying these terms will ensure the documentation is precise and comprehensive.

**actions**: 
 
 
1. **qa_lionagi**(query: Meaning of 'directive framework' in the context of chat interactions), 
 
2. **qa_lionagi**(query: Meaning of 'Unit' class in the context of lionagi.core.unit), 
 
3. **qa_lionagi**(query: Meaning of 'strip_lower' function in the context of lionagi.libs.ln_convert)

**action_required**: True

**extension_required**: True

**action_response**: 
 
 
1. **qa_lionagi**(query: Meaning of 'Unit' class in the context of lionagi.core.unit) 
 
 The 'Unit' class in the context of lionagi.core.unit is a specialized class that extends both the 'Directive' and 'DirectiveMixin' classes. It is designed to handle various operations related to directives within the system. The class initializes with parameters such as branch, imodel, template, and rulebook, and sets up a validator for rule enforcement. It also includes an asynchronous 'chat' method that facilitates communication and interaction, incorporating various parameters and options to customize its behavior., 
 
2. **qa_lionagi**(query: Meaning of 'strip_lower' function in the context of lionagi.libs.ln_convert) 
 
 The 'strip_lower' function in the context of lionagi.libs.ln_convert is likely a utility function that processes a string by stripping any leading or trailing whitespace and converting the string to lowercase. This is inferred from its usage in the 'get_input_output_fields' function, where it is applied to each input and output field to ensure they are in a standardized format., 
 
3. **qa_lionagi**(query: Meaning of 'strip_lower' function in the context of lionagi.libs.ln_convert) 
 
 The 'strip_lower' function in the context of lionagi.libs.ln_convert is likely a utility function that processes a string by stripping any leading or trailing whitespace and converting the string to lowercase. This is inferred from its usage in the 'get_input_output_fields' function, where it is applied to each input and output field to ensure they are in a standardized format., 
 
4. **qa_lionagi**(query: Meaning of 'directive framework' in the context of chat interactions) 
 
 In the context of chat interactions, the 'directive framework' refers to a structured system for managing and executing instructions or commands. It involves handling various aspects such as the sender and recipient of the instruction, the context in which the instruction is performed, and the tools or rulebooks used for validation and execution. This framework ensures that chat interactions are processed in an organized manner, allowing for additional instructions, context-specific actions, and the use of tools or models to achieve the desired outcomes.

**action_performed**: True

**available_tools**: ['qa_lionagi']

**answer**: 
 ## Class: `DirectiveMixin`

**Description**:
`DirectiveMixin` is a mixin class designed to handle chat interactions within the directive framework. It provides methods for processing instructions with various parameters and options, managing retries, timeouts, concurrency, and validation rules.

### Method: `chat`

**Signature**:
```python
async def chat(
    self,
    instruction: str = None,
    context: Any = None,
    system: str = None,
    sender: str = 'user',
    recipient: str = 'branch.ln_id',
    branch: Branch = None,
    requested_fields: dict[str, str] = None,
    form: Any = None,
    tools: bool = False,
    invoke_tool: bool = True,
    return_form: bool = True,
    strict: bool = False,
    rulebook: Any = None,
    imodel: iModel = None,
    clear_messages: bool = False,
    use_annotation: bool = True,
    retries: int = 3,
    delay: float = 0,
    backoff_factor: float = 1,
    default: Any = None,
    timeout: float = None,
    timing: bool = False,
    return_branch: bool = False,
    **kwargs: Any
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Additional instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `system` (str, optional): Optionally swap the system message.
- `sender` (str, optional): Sender of the instruction, default is 'user'.
- `recipient` (str, optional): Recipient of the instruction, default is 'branch.ln_id'.
- `branch` (Branch, optional): The branch to use for the instruction.
- `requested_fields` (dict[str, str], optional): Fields to request from the context.
- `form` (Any, optional): Form to create instruction from, default is None.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `invoke_tool` (bool, optional): Whether to invoke the tool when function calling, default is True.
- `return_form` (bool, optional): Whether to return the form if a form is passed in, otherwise return a dict/str.
- `strict` (bool, optional): Whether to strictly enforce rule validation, default is False.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `clear_messages` (bool, optional): Whether to clear previous messages, default is False.
- `use_annotation` (bool, optional): Whether to use annotation as rule qualifier, default is True (needs rulebook if False).
- `retries` (int, optional): Number of retries if failed, default is 3.
- `delay` (float, optional): Number of seconds to delay before retrying, default is 0.
- `backoff_factor` (float, optional): Exponential backoff factor, default is 1 (no backoff).
- `default` (Any, optional): Default value to return if all retries failed.
- `timeout` (float, optional): Timeout for the call, default is None (no timeout).
- `timing` (bool, optional): If True, will return a tuple (output, duration), default is False.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `**kwargs` (Any): Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the chat processing, format determined by the `return_form` parameter.

**Exceptions Raised**:
- `ValueError`: If an invalid combination of parameters is provided.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.chat(instruction='Hello', context={'data': 'example'})
print(result)
```

### Method: `direct`

**Signature**:
```python
async def direct(
    self,
    *,
    instruction: str = None,
    context: Any = None,
    form: Any = None,
    branch: Branch = None,
    tools: bool = None,
    return_branch: bool = False,
    reason: bool = False,
    predict: bool = False,
    score: Any = None,
    select: Any = None,
    plan: Any = None,
    allow_action: bool = False,
    allow_extension: bool = False,
    max_extension: int = None,
    confidence: Any = None,
    score_num_digits: int = None,
    score_range: Any = None,
    select_choices: Any = None,
    plan_num_step: int = None,
    predict_num_sentences: int = None,
    imodel: iModel = None,
    system: str = None,
    rulebook: Any = None,
    directive: Any = None,
    **kwargs: Any
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `form` (Any, optional): Form to create instruction from, default is None.
- `branch` (Branch, optional): The branch to use for the instruction.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `reason` (bool, optional): Whether to provide reasoning, default is False.
- `predict` (bool, optional): Whether to make predictions, default is False.
- `score` (Any, optional): Score to use for evaluation.
- `select` (Any, optional): Selection criteria.
- `plan` (Any, optional): Plan to follow.
- `allow_action` (bool, optional): Whether to allow actions, default is False.
- `allow_extension` (bool, optional): Whether to allow extensions, default is False.
- `max_extension` (int, optional): Maximum number of extensions, default is None.
- `confidence` (Any, optional): Confidence level.
- `score_num_digits` (int, optional): Number of digits for score.
- `score_range` (Any, optional): Range for score.
- `select_choices` (Any, optional): Choices for selection.
- `plan_num_step` (int, optional): Number of steps in the plan.
- `predict_num_sentences` (int, optional): Number of sentences for prediction.
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `system` (str, optional): Optionally swap the system message.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `directive` (Any, optional): Directive to follow.
- `**kwargs` (Any): Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the direct processing.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.direct(instruction='Hello', context={'data': 'example'})
print(result)
```

### Improve

In [17]:
edit = """
provide documentation only: final documentation in md 
format of the module of interest, do not include other 
fields do not present in JSON format, only markdown format 
you asked a lot of good questions and got plenty answers, 
please integrate your conversation, be a lot more technical, you will 
be rewarded with 500 dollars for great work, and 
punished for subpar work, take a deep breath, you can do it
"""

final_output = await branch.chat(instruction=edit)

In [18]:
from IPython.display import Markdown

Markdown(final_output)

# DirectiveMixin API Documentation

## Class: `DirectiveMixin`

**Description**:
`DirectiveMixin` is a mixin class designed to handle chat interactions within the directive framework. It provides methods for processing instructions with various parameters and options, managing retries, timeouts, concurrency, and validation rules.

### Method: `chat`

**Signature**:
```python
async def chat(
    self,
    instruction: str = None,
    context: Any = None,
    system: str = None,
    sender: str = 'user',
    recipient: str = 'branch.ln_id',
    branch: Branch = None,
    requested_fields: dict[str, str] = None,
    form: Any = None,
    tools: bool = False,
    invoke_tool: bool = True,
    return_form: bool = True,
    strict: bool = False,
    rulebook: Any = None,
    imodel: iModel = None,
    clear_messages: bool = False,
    use_annotation: bool = True,
    retries: int = 3,
    delay: float = 0,
    backoff_factor: float = 1,
    default: Any = None,
    timeout: float = None,
    timing: bool = False,
    return_branch: bool = False,
    **kwargs: Any
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Additional instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `system` (str, optional): Optionally swap the system message.
- `sender` (str, optional): Sender of the instruction, default is 'user'.
- `recipient` (str, optional): Recipient of the instruction, default is 'branch.ln_id'.
- `branch` (Branch, optional): The branch to use for the instruction.
- `requested_fields` (dict[str, str], optional): Fields to request from the context.
- `form` (Any, optional): Form to create instruction from, default is None.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `invoke_tool` (bool, optional): Whether to invoke the tool when function calling, default is True.
- `return_form` (bool, optional): Whether to return the form if a form is passed in, otherwise return a dict/str.
- `strict` (bool, optional): Whether to strictly enforce rule validation, default is False.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `clear_messages` (bool, optional): Whether to clear previous messages, default is False.
- `use_annotation` (bool, optional): Whether to use annotation as rule qualifier, default is True (needs rulebook if False).
- `retries` (int, optional): Number of retries if failed, default is 3.
- `delay` (float, optional): Number of seconds to delay before retrying, default is 0.
- `backoff_factor` (float, optional): Exponential backoff factor, default is 1 (no backoff).
- `default` (Any, optional): Default value to return if all retries failed.
- `timeout` (float, optional): Timeout for the call, default is None (no timeout).
- `timing` (bool, optional): If True, will return a tuple (output, duration), default is False.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `**kwargs` (Any): Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the chat processing, format determined by the `return_form` parameter.

**Exceptions Raised**:
- `ValueError`: If an invalid combination of parameters is provided.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.chat(instruction='Hello', context={'data': 'example'})
print(result)
```

### Method: `direct`

**Signature**:
```python
async def direct(
    self,
    *,
    instruction: str = None,
    context: Any = None,
    form: Any = None,
    branch: Branch = None,
    tools: bool = None,
    return_branch: bool = False,
    reason: bool = False,
    predict: bool = False,
    score: Any = None,
    select: Any = None,
    plan: Any = None,
    allow_action: bool = False,
    allow_extension: bool = False,
    max_extension: int = None,
    confidence: Any = None,
    score_num_digits: int = None,
    score_range: Any = None,
    select_choices: Any = None,
    plan_num_step: int = None,
    predict_num_sentences: int = None,
    imodel: iModel = None,
    system: str = None,
    rulebook: Any = None,
    directive: Any = None,
    **kwargs: Any
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `form` (Any, optional): Form to create instruction from, default is None.
- `branch` (Branch, optional): The branch to use for the instruction.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `reason` (bool, optional): Whether to provide reasoning, default is False.
- `predict` (bool, optional): Whether to make predictions, default is False.
- `score` (Any, optional): Score to use for evaluation.
- `select` (Any, optional): Selection criteria.
- `plan` (Any, optional): Plan to follow.
- `allow_action` (bool, optional): Whether to allow actions, default is False.
- `allow_extension` (bool, optional): Whether to allow extensions, default is False.
- `max_extension` (int, optional): Maximum number of extensions, default is None.
- `confidence` (Any, optional): Confidence level.
- `score_num_digits` (int, optional): Number of digits for score.
- `score_range` (Any, optional): Range for score.
- `select_choices` (Any, optional): Choices for selection.
- `plan_num_step` (int, optional): Number of steps in the plan.
- `predict_num_sentences` (int, optional): Number of sentences for prediction.
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `system` (str, optional): Optionally swap the system message.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `directive` (Any, optional): Directive to follow.
- `**kwargs` (Any): Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the direct processing.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.direct(instruction='Hello', context={'data': 'example'})
print(result)
```

### check extension forms

In [19]:
if hasattr(form, "extension_forms"):
    for i in form.extension_forms:
        i.display()

**task**: 
 Follow the prompt and provide the necessary output.
- Additional instruction: N/A
- Additional context: N/A
- Reason and prepare actions with GIVEN TOOLS ONLY.


**reason**: 
 Let's think step by step. The previous actions have provided the necessary information about the 'directive framework', 'Unit' class, and 'strip_lower' function. Now, I will proceed to write the API documentation for the provided code.

**actions**: 

**action_required**: False

**available_tools**: ['qa_lionagi']

**answer**: 
 ## Class: `DirectiveMixin`

**Description**:
`DirectiveMixin` is a mixin class designed to handle chat interactions within the directive framework. It provides methods for processing instructions with various parameters and options, managing retries, timeouts, concurrency, and validation rules.

### Method: `chat`

**Signature**:
```python
async def chat(
    self,
    instruction: str = None,
    context: Any = None,
    system: str = None,
    sender: str = 'user',
    recipient: str = 'branch.ln_id',
    branch: Branch = None,
    requested_fields: dict[str, str] = None,
    form: Any = None,
    tools: bool = False,
    invoke_tool: bool = True,
    return_form: bool = True,
    strict: bool = False,
    rulebook: Any = None,
    imodel: iModel = None,
    clear_messages: bool = False,
    use_annotation: bool = True,
    retries: int = 3,
    delay: float = 0,
    backoff_factor: float = 1,
    default: Any = None,
    timeout: float = None,
    timing: bool = False,
    return_branch: bool = False,
    **kwargs: Any
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Additional instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `system` (str, optional): Optionally swap the system message.
- `sender` (str, optional): Sender of the instruction, default is 'user'.
- `recipient` (str, optional): Recipient of the instruction, default is 'branch.ln_id'.
- `branch` (Branch, optional): The branch to use for the instruction.
- `requested_fields` (dict[str, str], optional): Fields to request from the context.
- `form` (Any, optional): Form to create instruction from, default is None.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `invoke_tool` (bool, optional): Whether to invoke the tool when function calling, default is True.
- `return_form` (bool, optional): Whether to return the form if a form is passed in, otherwise return a dict/str.
- `strict` (bool, optional): Whether to strictly enforce rule validation, default is False.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `clear_messages` (bool, optional): Whether to clear previous messages, default is False.
- `use_annotation` (bool, optional): Whether to use annotation as rule qualifier, default is True (needs rulebook if False).
- `retries` (int, optional): Number of retries if failed, default is 3.
- `delay` (float, optional): Number of seconds to delay before retrying, default is 0.
- `backoff_factor` (float, optional): Exponential backoff factor, default is 1 (no backoff).
- `default` (Any, optional): Default value to return if all retries failed.
- `timeout` (float, optional): Timeout for the call, default is None (no timeout).
- `timing` (bool, optional): If True, will return a tuple (output, duration), default is False.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `**kwargs` (Any): Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the chat processing, format determined by the `return_form` parameter.

**Exceptions Raised**:
- `ValueError`: If an invalid combination of parameters is provided.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.chat(instruction='Hello', context={'data': 'example'})
print(result)
```

### Method: `direct`

**Signature**:
```python
async def direct(
    self,
    *,
    instruction: str = None,
    context: Any = None,
    form: Any = None,
    branch: Branch = None,
    tools: bool = None,
    return_branch: bool = False,
    reason: bool = False,
    predict: bool = False,
    score: Any = None,
    select: Any = None,
    plan: Any = None,
    allow_action: bool = False,
    allow_extension: bool = False,
    max_extension: int = None,
    confidence: Any = None,
    score_num_digits: int = None,
    score_range: Any = None,
    select_choices: Any = None,
    plan_num_step: int = None,
    predict_num_sentences: int = None,
    imodel: iModel = None,
    system: str = None,
    rulebook: Any = None,
    directive: Any = None,
    **kwargs: Any
) -> Any:
```

**Parameters**:
- `instruction` (str, optional): Instruction to process.
- `context` (Any, optional): Context to perform the instruction on.
- `form` (Any, optional): Form to create instruction from, default is None.
- `branch` (Branch, optional): The branch to use for the instruction.
- `tools` (bool, optional): Tools to use, use True to consider all tools, no tools by default.
- `return_branch` (bool, optional): Whether to return the branch after processing, default is False.
- `reason` (bool, optional): Whether to provide reasoning, default is False.
- `predict` (bool, optional): Whether to make predictions, default is False.
- `score` (Any, optional): Score to use for evaluation.
- `select` (Any, optional): Selection criteria.
- `plan` (Any, optional): Plan to follow.
- `allow_action` (bool, optional): Whether to allow actions, default is False.
- `allow_extension` (bool, optional): Whether to allow extensions, default is False.
- `max_extension` (int, optional): Maximum number of extensions, default is None.
- `confidence` (Any, optional): Confidence level.
- `score_num_digits` (int, optional): Number of digits for score.
- `score_range` (Any, optional): Range for score.
- `select_choices` (Any, optional): Choices for selection.
- `plan_num_step` (int, optional): Number of steps in the plan.
- `predict_num_sentences` (int, optional): Number of sentences for prediction.
- `imodel` (iModel, optional): Optionally swappable iModel for the commands, otherwise self.branch.imodel.
- `system` (str, optional): Optionally swap the system message.
- `rulebook` (Any, optional): The rulebook to use for validation, default is None (uses default rulebook).
- `directive` (Any, optional): Directive to follow.
- `**kwargs` (Any): Additional keyword arguments for further customization.

**Return Values**:
- `Any`: The result of the direct processing.

**Usage Examples**:
```python
# Example 1: Basic usage with instruction and context
result = await self.direct(instruction='Hello', context={'data': 'example'})
print(result)
```

### check actions

In [20]:
print(f"The task invoked query engine {len(pile.query_response)} times.")

for i in pile.query_response:
    print(i.response)

The task invoked query engine 3 times.
The 'strip_lower' function in the context of lionagi.libs.ln_convert is likely a utility function that processes a string by stripping any leading or trailing whitespace and converting the string to lowercase. This is inferred from its usage in the 'get_input_output_fields' function, where it is applied to each input and output field to ensure they are in a standardized format.
In the context of chat interactions, the 'directive framework' refers to a structured system for managing and executing instructions or commands. It involves handling various aspects such as the sender and recipient of the instruction, the context in which the instruction is performed, and the tools or rulebooks used for validation and execution. This framework ensures that chat interactions are processed in an organized manner, allowing for additional instructions, context-specific actions, and the use of tools or models to achieve the desired outcomes.
The 'Unit' class in